GeoViews is designed to make full use of multidimensional gridded datasets stored in netCDF or other common formats, via the xarray and iris interfaces in HoloViews. This notebook will demonstrate how to load data using both of these data backends, along with some of their individual quirks. The data used in this notebook was originally shipped as part of the [``SciTools/iris-sample-data``](https://github.com/SciTools/iris-sample-data) repository, but a smaller netCDF file is included as part of the GeoViews so that it can be used with xarray as well.

In [ ]:
import iris
import numpy as np
import xarray as xr
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf

from cartopy import crs
from geoviews import opts

gv.extension('matplotlib')

gv.output(size=150)

## Loading our data

In this notebook we will primarily be working with xarray, but we will also load the same data using iris so that we can demonstrate that the two data backends are nearly equivalent.

#### XArray

As a first step we simply load the data using the ``open_dataset`` method xarray provides and have a look at the repr to get an overview what is in this dataset:

In [ ]:
xr_ensemble = xr.open_dataset('../data/ensemble.nc').load()
xr_ensemble

#### Iris

Similarly we can load the same dataset using Iris' ``load_cube`` function and get a similar overview using the ``summary`` method.

In [ ]:
iris.FUTURE.netcdf_promote=True
iris_ensemble = iris.load_cube('../data/ensemble.nc')
print(iris_ensemble.summary())

Describing the differences between these two libraries is well beyond the scope of this tutorial, but you can see from the summaries that the two libraries deal differently with both the bounds and with the actual data variables. Iris cubes support only a single data variable, while an xarray dataset can have any number of variables. In this case we are only interested in the ``surface_temperature`` dimension, indexed by ``longitude``, ``latitude`` and ``time``.

We can easily express this interest by wrapping the data in a GeoViews ``Dataset`` Element and declaring the key dimensions (``kdims``) and value dimensions (``vdims``). Note that the Iris interface is much smarter in the way it extracts the dimensions, so usually you will not have to supply them explicitly.

In [ ]:
kdims = ['time', 'longitude', 'latitude']
vdims = ['surface_temperature']

xr_dataset = gv.Dataset(xr_ensemble, kdims=kdims, vdims=vdims)
iris_dataset = gv.Dataset(iris_ensemble, kdims=kdims, vdims=vdims)

Now we can compare the repr of the two Elements:

In [ ]:
print(repr(xr_dataset))
print(repr(iris_dataset))

Despite appearing identical, there are some internal differences, such as in the data types. xarray uses NumPy datetime64 types for dates, while iris will use simple floats:

In [ ]:
print("XArray time type: %s" % xr_dataset.get_dimension_type('time'))
print("Iris time type: %s" % iris_dataset.get_dimension_type('time'))

To improve the formatting of dates on the xarray dataset we can set the formatter for datetime64 types:

In [ ]:
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

The other major differences in the way iris cubes are handled are in deducing various bits of metadata including the coordinate system, units, and formatters. Otherwise the two Dataset Elements will behave largely the same.

For either data backend, the `Dataset` object is not yet visualizable, because we have not chosen which dimensions to map onto which axes of a plot.

# A Simple example

To visualize the datasets, in a single line of code we can specify that we want to view it as a collection of Images indexed by longitude and latitude (a HoloViews ``HoloMap`` of ``gv.Image`` elements):

In [ ]:
xr_dataset.to(gv.Image, ['longitude', 'latitude'])

You can see that the `time` dimension was automatically mapped to a slider, because we did not map it onto one of the other available dimensions (x, y, or color, in this case). You can drag the slider to view the surface temperature at different times. The example would work just the same for the ``iris_dataset``.

Now let us load a cube showing the pre-industrial air temperature:

In [ ]:
pre_industrial = xr.open_dataset('../data/pre-industrial.nc').load()

air_temperature = gv.Dataset(pre_industrial, ['longitude', 'latitude'], 'air_temperature')
air_temperature

Note that we have the ``air_temperature`` available over ``longitude`` and ``latitude`` but *not* the ``time`` dimensions. As a result, this cube is a single frame (at right below) when visualized as an ``Image``:

In [ ]:
gv.Image(air_temperature)

The following more complicated example shows how complex interactive plots can be generated with relatively little code, and also demonstrates how different HoloViews elements can be combined together. In the following visualization, the black dot denotes a specific longitude, latitude location *(0,10)*, and the curve is a sample of the ``surface_temperature`` at that location.  The curve is unaffected by the `time` slider because it already lays out time along the x axis:

In [ ]:
temp_curve = hv.Curve(xr_dataset.select(longitude=0, latitude=10), kdims=['time'])
temp_map = xr_dataset.to(gv.Image,['longitude', 'latitude']) * gv.Points([(0,10)])
(temp_map + temp_curve).opts(
    opts.Curve(aspect=2, xticks=4, xrotation=15),
    opts.Points(color='k', global_extent=True))

## Overlaying data and normalization

Let's view the surface temperatures together with the global coastline:

In [ ]:
(xr_dataset.to.image(['longitude', 'latitude']) * gf.coastline).opts(
    opts.Image(projection=crs.Geostationary(), cmap='Greens', xaxis=None, yaxis=None))

Notice that every frame individually uses the full dynamic range of the Greens color map. This is because normalization is set to ``+framewise`` at the top of the notebook, which means every frame is normalized independently. This sort of normalization can be computed on an as-needed basis, using whatever values are found in the current data being shown in a given frame, but it won't let you see how different frames compare to each other.

To control normalization, we need to decide on the normalization limits. Let's see the maximum temperature in the cube, and use it to set a normalization range by using the redim method:

In [ ]:
max_surface_temp = xr_dataset.range('surface_temperature')[1]
print(max_surface_temp)
srfc_temps = xr_dataset.redim.range(surface_temperature=(300, max_surface_temp)).to(gv.Image,['longitude', 'latitude'])

(srfc_temps * gf.coastline).opts(
    opts.Image(projection=crs.Geostationary(), cmap='Greens', xaxis=None, yaxis=None))

By specifying the normalization range we can reveal different aspects of the data. In the example above we can see a cooling effect over time as the dark green areas close to the top of the normalization range (317K) vanish. Values outside this range are clipped to the ends of the color map.

Lastly, here is a demo of a conversion from ``surface_temperature`` to ``FilledContours``:

In [ ]:
xr_dataset.to(gv.FilledContours,['longitude', 'latitude']) * gf.coastline

As you can see, it's quite simple to expose any data you like from your Iris cube or xarray, easily and flexibly creating interactive or static visualizations.